### Import library

In [1]:
!pip install websocket-client



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import websocket # this will be added with your pip installation earlier
import json

def on_message(ws, message):

    # This function will be called when new messages arrive at the WebSocket
    json_message = json.loads(message)
    candle = json_message['k']
    print("New candle received:")

    print(f"Time: {candle['t']}")  # Open time of the candle
    print(f"Open: {candle['o']}")
    print(f"Close: {candle['c']}")
    print(f"High: {candle['h']}")
    print(f"Low: {candle['l']}")
    print(f"Volume: {candle['v']}")
    print('-' * 50)

def on_error(ws, error):
    print(f"Error: {error}")

def on_close(ws, close_status_code, close_msg):
    print("### closed ###")
    print("Close status code: ", close_status_code)
    print("Close message: ", close_msg)

def on_open(ws):

    # This function runs when the WebSocket opens.
    print("WebSocket opened")

    # Subscribe to the 1m candlestick chart for BTC/USDT
    subscribe_message = {
        "method": "SUBSCRIBE",
        "params": [
            "nearusdt@kline_1s"
        ],
        "id": 1
    }

    ws.send(json.dumps(subscribe_message))

In [9]:
import json
import websocket
import datetime

def ws_trades(): 
    socket = f'wss://stream.binance.com:9443/ws/nearusdt@trade'

    def on_message(wsapp,message):  
        json_message = json.loads(message)
        handle_trades(json_message)

    def on_error(wsapp,error):
        print(error)

    wsapp = websocket.WebSocketApp(socket, on_message=on_message, on_error=on_error)
    wsapp.run_forever()
    
def handle_trades(json_message):
    date_time = datetime.datetime.fromtimestamp(json_message['E']/1000).strftime('%Y-%m-%d %H:%M:%S')
    print("SYMBOL: "+json_message['s'])
    print("PRICE: "+json_message['p'])
    print("QTY: "+json_message['q'])
    print("TIMESTAMP: " + str(date_time))
    print("-----------------------")

In [10]:
        message_old = int(df_save_vol_vol.loc[df_save_vol_vol['symbol'] == event.symbol, 'message_old'].values[0])
        tick_num1 = int(df_save_vol_vol.loc[df_save_vol_vol['symbol'] == event.symbol, 'tick_num1'].values[0])
        stats = df_save_vol_vol.loc[df_save_vol_vol['symbol'] == event.symbol, 'stats'].values[0]
        open_price = (df_save_vol_vol.loc[df_save_vol_vol['symbol'] == event.symbol, 'open_price'].values[0])
        high_price = (df_save_vol_vol.loc[df_save_vol_vol['symbol'] == event.symbol, 'high_price'].values[0])
        low_price = (df_save_vol_vol.loc[df_save_vol_vol['symbol'] == event.symbol, 'low_price'].values[0])
        prev_price  = (df_save_vol_vol.loc[df_save_vol_vol['symbol'] == event.symbol, 'prev_price'].values[0])
        close_price = (df_save_vol_vol.loc[df_save_vol_vol['symbol'] == event.symbol, 'close_price'].values[0])
        
        records = event.__dict__
        new_dict = {key: value for key, value in records.items() if key != '__evaluated_fields__'}
        # Nhận transactions mới vào và tính
        bar,new_stats,new_open_price, new_high_price,new_low_price,new_tick_nums,new_prev_price,new_close_price = message_to_volume_bar(new_dict,stats,open_price,high_price,low_price,tick_num1,prev_price,close_price,event.symbol)
        print("new_stats:", new_stats)
        print("new_open_price:", new_open_price)

        # nếu là bar mới thì mới gửi vào topic còn không thì kệ nó 
        if  bar is not None and int(bar.tick_num) != message_old :
            #message_old = int(bar.tick_num)
            #tick_num1 = int(new_tick_nums)
            #stats = new_stats
            #open_price = new_open_price
            #high_price = new_high_price
            #low_price = new_low_price
            #prev_price = new_prev_price
            # close_price = new_close_price

            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'message_old'] = int(bar.tick_num)
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'tick_num1'] = int(new_tick_nums)
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'stats'] = [new_stats]
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'open_price'] = new_open_price
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'high_price'] = new_high_price
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'low_price'] =  new_low_price
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'prev_price'] = new_prev_price
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'close_price'] = new_close_price
            # print(df_save_vol)

            records1 = bar.__dict__
            new_bar = {key: value for key, value in records1.items() if key != '__evaluated_fields__'}
            new_bar['Symbol'] = event.symbol
            top_df = pd.DataFrame.from_records(topPrice)
            top_df = top_df[top_df['Symbol'] == event.symbol]
            top_df['Time'] = pd.to_datetime(top_df['Time'])
            # Example target datetime
            x = pd.to_datetime(new_bar['date_time'])
            # Compute the absolute difference between each date_time and x
            top_df['time_diff'] = (top_df['Time'] - x).abs()
            # Sort the dataframe by the time difference
            top_df_sorted = top_df.sort_values(by='time_diff')
            # Get the row(s) with the smallest time difference
            closest_row = top_df_sorted.iloc[0]
            # Extract the time value(s) of the closest row
            closest_time_value = closest_row['Time']
            bid_price = closest_row['bid_price']
            ask_price = closest_row['ask_price']
            ask_volume = closest_row['volume_ask']
            bid_volume = closest_row['volume_bid']
            if new_bar['close'] <= bid_price :
                new_bar['Label_Chủ_Động'] = 'B'
                new_bar['Price_Chủ_Động'] = bid_price
                new_bar['Volume_Chủ_Động'] = ask_volume
            elif new_bar['close'] >= ask_price:
                new_bar['Label_Chủ_Động'] = 'M'  
                new_bar['Price_Chủ_Động'] = ask_price
                new_bar['Volume_Chủ_Động'] = bid_volume

            print(f'Send_Volume_Bar{new_bar}')          
            save_to_db(df=pd.DataFrame([new_bar]), table='volume_bar_real_time_message', index=False)
            #await Volumne_bars_topic.send(value = bar,key = event.symbol)
        elif  bar is None :
            # tick_num1 = int(new_tick_nums)
            # stats = new_stats
            # open_price = new_open_price
            # high_price = new_high_price
            # low_price = new_low_price
            # prev_price = new_prev_price
            # close_price = new_close_price
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'tick_num1'] = int(new_tick_nums)
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'stats'] = [new_stats]
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'open_price'] = [new_open_price]
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'high_price'] = [new_high_price]
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'low_price'] = [new_low_price]
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'prev_price'] = [new_prev_price]
            df_save_vol.loc[df_save_vol['symbol'] == event.symbol,'close_price'] = [new_close_price]

--- request header ---
GET /ws/nearusdt@trade HTTP/1.1
Upgrade: websocket
Host: stream.binance.com:9443
Origin: http://stream.binance.com:9443
Sec-WebSocket-Key: x6i+HFHXsiD3Vs4Pdy4tnw==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Thu, 23 May 2024 08:35:32 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: eZeFzZA8fFxo7J1ZmIfGRNqL2/E=
-----------------------


SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 171.20000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 28.80000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 265.90000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 53.60000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 1180.50000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 214.00000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 9.70000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 50.00000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.06200000
QTY: 50.00000000
TIMESTAMP: 2024-05-23 15:35:41
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.0620000

send: b'\x8a\x8dtBw\xe1EuF\xd7@wD\xd2MuB\xd3C'


SYMBOL: NEARUSDT
PRICE: 8.05700000
QTY: 3.50000000
TIMESTAMP: 2024-05-23 15:36:38
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05700000
QTY: 3.50000000
TIMESTAMP: 2024-05-23 15:36:40
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05600000
QTY: 8.20000000
TIMESTAMP: 2024-05-23 15:36:40
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05700000
QTY: 3.50000000
TIMESTAMP: 2024-05-23 15:36:44
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05700000
QTY: 1.80000000
TIMESTAMP: 2024-05-23 15:36:46
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05700000
QTY: 1.70000000
TIMESTAMP: 2024-05-23 15:36:46
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05700000
QTY: 3.50000000
TIMESTAMP: 2024-05-23 15:36:48
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05700000
QTY: 3.50000000
TIMESTAMP: 2024-05-23 15:36:53
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05600000
QTY: 91.80000000
TIMESTAMP: 2024-05-23 15:36:54
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.05400000
QTY: 4.000

send: b'\x8a\x8dV\xa7;_g\x90\nib\x92\x08ja\x90\x0eib'


...

send: b'\x8a\x8d\x87U\xfe^\xb6b\xcfh\xb3`\xcaj\xb0b\xc9h\xb3'


SYMBOL: NEARUSDT
PRICE: 8.02500000
QTY: 0.80000000
TIMESTAMP: 2024-05-23 15:54:44
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02400000
QTY: 85.40000000
TIMESTAMP: 2024-05-23 15:54:44
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02400000
QTY: 106.50000000
TIMESTAMP: 2024-05-23 15:54:44
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02400000
QTY: 50.00000000
TIMESTAMP: 2024-05-23 15:54:44
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02400000
QTY: 100.00000000
TIMESTAMP: 2024-05-23 15:54:44
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02400000
QTY: 278.30000000
TIMESTAMP: 2024-05-23 15:54:44
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02500000
QTY: 3.30000000
TIMESTAMP: 2024-05-23 15:54:47
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02500000
QTY: 15.00000000
TIMESTAMP: 2024-05-23 15:54:48
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02500000
QTY: 184.60000000
TIMESTAMP: 2024-05-23 15:54:55
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02500000


send: b'\x8a\x8d\xea\xeb\xec\x04\xdb\xdc\xdd2\xde\xde\xd82\xdf\xdc\xd45\xdb'


SYMBOL: NEARUSDT
PRICE: 8.02400000
QTY: 1.00000000
TIMESTAMP: 2024-05-23 15:57:42
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02400000
QTY: 48.60000000
TIMESTAMP: 2024-05-23 15:57:49
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02400000
QTY: 48.10000000
TIMESTAMP: 2024-05-23 15:57:49
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02500000
QTY: 16.80000000
TIMESTAMP: 2024-05-23 15:57:49
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02500000
QTY: 49.20000000
TIMESTAMP: 2024-05-23 15:57:49
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02500000
QTY: 1.00000000
TIMESTAMP: 2024-05-23 15:57:49
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02600000
QTY: 49.50000000
TIMESTAMP: 2024-05-23 15:57:52
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02600000
QTY: 82.40000000
TIMESTAMP: 2024-05-23 15:57:52
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02600000
QTY: 1.00000000
TIMESTAMP: 2024-05-23 15:57:52
-----------------------
SYMBOL: NEARUSDT
PRICE: 8.02700000
QTY: 

send: b'\x88\x82\xf3_\xcfk\xf0\xb7'
